In [39]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import statistics

import random
random.seed(10)


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
from scipy.stats import uniform
import scipy.stats as st
from scipy.stats import pointbiserialr
from scipy.stats import wilcoxon
from scipy.stats import kruskal



import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import GammaRegressor
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor


from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


# Investment Portfolio Data Management

[Business requirement] : Write a python script to create an output csv file as specified from the following 4 input csv files [balancesheet.csv, companystockinfo.csv, incomestatement.csv, fxrate.csv, forecast.csv] to create output file aggregating information of all individual stocks in one dataframe.  

The new output file contains information from 134 companies, extracted from 4 files. It selects specific values to calculate new summaries that assist portfolio managers in understanding stock performance more easily. This solution reduces the time needed for updates, recalculations, and data gathering after each company's financial results out or stock analyst forecasts.

# Data Assets


In [40]:
df_balance = pd.read_csv('/Users/phimonkaewilairat/Desktop/balancesheet.csv')
df_stock = pd.read_csv('/Users/phimonkaewilairat/Desktop/companystockinfo.csv')
df_income = pd.read_csv('/Users/phimonkaewilairat/Desktop/incomestatement.csv')
df_fxrate = pd.read_csv('/Users/phimonkaewilairat/Desktop/fxrate.csv')
df_forecast = pd.read_csv('/Users/phimonkaewilairat/Desktop/forecast.csv')
df_demo = pd.read_csv('/Users/phimonkaewilairat/Desktop/sample_output.csv')

# Ouput DataFrame Generation

In [41]:
list_output_columns = df_demo.columns.to_list()

In [42]:
list_output_columns = df_demo.columns.to_list()
dfout = pd.DataFrame(data = None, columns = list_output_columns)
dfout # this is our output dataframe to be filled 

,companyid,latestfinyear,roe fy-4,roe fy-3,roe fy-2,roe fy-1,roe fy,roe fy+1,roe fy+2,roe fy+3,5yr median roe,roa fy-4,roa fy-3,roa fy-2,roa fy-1,roa fy,roa fy+1,roa fy+2,roa fy+3,5yr median roa,netde fy,revenue fy-4,revenue fy-3,revenue fy-2,revenue fy-1,revenue fy,revenue fy+1,revenue fy+2,revenue fy+3,revenue 2yr cagr,revenue 3yr cagr


# Data Generation and Input to dfout

In [43]:
# Inserting company id and latestfinyear

dfout.companyid = df_stock.companyid
dfout.latestfinyear = df_stock.latestfinyear

In [44]:
# Inserting roe fy-4 to fy+3

list_roe = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    list_years = range(year-4, year+4, 1) # generates list of years for which features are required
    
#########################################################################################################
        

# Process for generating roe and roA - 

#Generation of both variables requires profit, equity and revenue data from multiple sheets. 
#When extracting said data for years post the latestfinyear.
#The data needs to be extracted from the forecast dataframe rather than original income or balance sheet dataframe. 


        
    for i in list_years:
        
        search_year = i # year for which values will be lookedup in other dataframes
        
        ############################################################
        ############################################################
        # Extracting variables values for roe 
        ############################################################
        ############################################################
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting net_profit
        
        
        dataframe = df_income
        net_profit = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].netprofit
        
        net_profit_status = net_profit.empty
        
        if net_profit_status == True:
            
            net_profit = 0
            
        net_profit = float(net_profit)
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity
        
        dataframe = df_balance
        total_equity = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totequity
        
        total_equity_status = total_equity.empty
        
        if total_equity_status == True:
            
            total_equity = 0
            
        total_equity = float(total_equity)

        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity_prev_year
        
        dataframe = df_balance
        total_equity_prev_year = dataframe[(dataframe.companyid == companyid) & (dataframe.year == (search_year - 1))].totequity
        
        total_equity_prev_year_status = total_equity_prev_year.empty
        
        if total_equity_prev_year_status == True:
            
            total_equity_prev_year = 0
            
        total_equity_prev_year = float(total_equity_prev_year)
            
        
        ############################################################
        ############################################################
        # Generating the roe
        ############################################################
        ############################################################
        
        # roe = 100 * (netprofit(year) / ((totalequity(year)+totalequity(year-1))/2))
        
        
        if net_profit_status == False:
            
            num = net_profit
            den = (total_equity + total_equity_prev_year) * 0.5
        
            roe = (num/den) * 100
            
        if net_profit_status == True:
            
            dataframe = df_forecast
            roe = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].roe
            
            if roe.empty == True:
                roe = 0
            
            roe = float(roe) # required for all roe directly taken from df_forecasts
        
        ############################################################
        ############################################################
        # Output operations
        ############################################################
        ############################################################
        
        
        '''
        print('search_year', search_year)
        print('companyid',companyid)
        print(list_years)
        print(search_year)
        print(roe)
        print('Roe generated for year', search_year)
        print('companyid', companyid)
        print('Roe generated for year', search_year)     
        print('toteqprev', total_equity_prev_year)
        print(search_year)
        print('companyid', companyid)
        print(search_year)
        print(roe)
        '''
        
        # Inserting roe into dfout
        
        list_roe.append(roe)
        

########## Dividing roe list into chunks of 8 

# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
 
# How many elements each
# list should have
n = 8
 
list_roe_chunks = list(divide_chunks(list_roe, n))  


########## Inserting roe values in dfout

for i in dfout.index:
    
    chunk = list_roe_chunks[i]
    
    dfout['roe fy-4'][i] = chunk[0]
    dfout['roe fy-3'][i] = chunk[1]
    dfout['roe fy-2'][i] = chunk[2]
    dfout['roe fy-1'][i] = chunk[3]
    dfout['roe fy'][i]   = chunk[4]
    dfout['roe fy+1'][i] = chunk[5]
    dfout['roe fy+2'][i] = chunk[6]
    dfout['roe fy+3'][i] = chunk[7]
    
        
            
        

In [37]:
############ Inserting roa fy-4 to fy+3

list_roa = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    list_years = range(year-4, year+4, 1) # generates list of years for which features are required
    
#########################################################################################################
        

# Process for generating roe and roA - 

#Generation of both variables requires profit, equity and revenue data from multiple sheets. 
#When extracting said data for years post the latestfinyear.
#The data needs to be extracted from the forecast dataframe rather than original income or balance sheet dataframe. 


        
    for i in list_years:
        
        search_year = i # year for which values will be lookedup in other dataframes
        
        ############################################################
        ############################################################
        # Extracting variables values for roe 
        ############################################################
        ############################################################
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting net_profit
        
        
        dataframe = df_income
        net_profit = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].netprofit
        
        net_profit_status = net_profit.empty
        
        if net_profit_status == True:
            
            net_profit = 0
            
        net_profit = float(net_profit)
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity
        
        dataframe = df_balance
        total_assets = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totassets
        
        total_assets_status = total_assets.empty
        
        if total_assets_status == True:
            
            total_assets = 0
            
        total_assets = float(total_assets)

        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity_prev_year
        
        dataframe = df_balance
        total_assets_prev_year = dataframe[(dataframe.companyid == companyid) & (dataframe.year == (search_year - 1))].totassets
        
        total_assets_prev_year_status = total_assets_prev_year.empty
        
        if total_assets_prev_year_status == True:
            
            total_assets_prev_year = 0
            
        total_assets_prev_year = float(total_assets_prev_year)
            
        
        ############################################################
        ############################################################
        # Generating the roe
        ############################################################
        ############################################################
        
        # roe = 100 * (netprofit(year) / ((totalassets(year)+totalassets(year-1))/2))
        
        
        if net_profit_status == False:
            
            num = net_profit
            den = (total_assets + total_assets_prev_year) * 0.5
        
            roa = (num/den) * 100
            
        if net_profit_status == True:
            
            dataframe = df_forecast
            roa = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].roa
            
            if roa.empty == True:
                roa = 0
            
            roa = float(roa) # required for all roe directly taken from df_forecasts
        
        ############################################################
        ############################################################
        # Output operations
        ############################################################
        ############################################################
        
        
        '''
        
        print('companyid',companyid)
        print(list_years)
        print('Roe generated for year', search_year)
        print('companyid', companyid)
        print('Roe generated for year', search_year)     
        print('toteqprev', total_equity_prev_year)
        
        print('companyid', companyid)
        print(search_year)
        
        print('companyid:', companyid)
        print('latestinfyear:', year)
        print('search_year:', search_year)
        print('roa:',roa)
        print('\n')
        
        
        '''
        
        # Inserting roe into dfout
       
        list_roa.append(roa)
        
     

######### Dividing roa list into chunks of 8 

# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
 
# How many elements each
# list should have
n = 8
 
list_roa_chunks = list(divide_chunks(list_roa, n))  


########## Inserting roe values in dfout

for i in dfout.index:
    
    chunk = list_roa_chunks[i]
    
    dfout['roa fy-4'][i] = chunk[0]
    dfout['roa fy-3'][i] = chunk[1]
    dfout['roa fy-2'][i] = chunk[2]
    dfout['roa fy-1'][i] = chunk[3]
    dfout['roa fy'][i]   = chunk[4]
    dfout['roa fy+1'][i] = chunk[5]
    dfout['roa fy+2'][i] = chunk[6]
    dfout['roa fy+3'][i] = chunk[7]
   

In [45]:
### Generating 5ry median roe and roa

# missing value imputation is required for roe and roa columns
# then columns need to be typecasted
# only then numpy computations will be enabled

### imputation
roe_columns = ['roe fy-4', 'roe fy-3', 'roe fy-2', 'roe fy-1', 'roe fy', 'roe fy+1', 'roe fy+2', 'roe fy+3']
roa_columns = ['roa fy-4', 'roa fy-3', 'roa fy-2', 'roa fy-1', 'roa fy', 'roa fy+1', 'roa fy+2', 'roa fy+3']

# imputation value is set to 0.01 to recognize the observations and not affect statistical computation
#dfout[roe_columns] = dfout[roe_columns].fillna(value = 0.01)
#dfout[roa_columns] = dfout[roa_columns].fillna(value = 0.01)

### type casting
# successful imputation will automatically typecast the columns
# if not, then manual typcasting will be required

for i in dfout.index:
    
    # inserting median for roe
    list_values = dfout[['roe fy-4', 'roe fy-3', 'roe fy-2', 'roe fy-1', 'roe fy']].iloc[i].to_list()
    median = statistics.median(list_values)
    dfout['5yr median roe'][i] = median
    
    # inserting median for roa
    list_values = dfout[['roa fy-4', 'roa fy-3', 'roa fy-2', 'roa fy-1', 'roa fy']].iloc[i].to_list()
    median = statistics.median(list_values)
    dfout['5yr median roa'][i] = median

In [46]:
### generating revenue columns
# similar procedure to roe and roa


list_revenue = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    list_years = range(year-4, year+4, 1) # generates list of years for which features are required
    
#########################################################################################################
        

# Process for generating roe and roA - 

#Generation of both variables requires profit, equity and revenue data from multiple sheets. 
#When extracting said data for years post the latestfinyear.
#The data needs to be extracted from the forecast dataframe rather than original income or balance sheet dataframe. 


        
    for i in list_years:
        
        search_year = i # year for which values will be lookedup in other dataframes
        
        ############################################################
        ############################################################
        # Extracting variables values for roe 
        ############################################################
        ############################################################
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting net_profit
        
        
        dataframe = df_income
        revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue
        
        revenue_status = revenue.empty
        
        if revenue_status == True: # to check if revenue was missing in df_income
            
            # if revenue missing in df_income; collect revenue from forecast
            
            dataframe = df_forecast
            revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue

            if (revenue.empty == True):
                
                revenue = 0.01 # if revenue is not found in forecast, value/observation is an indentifier
            
        revenue = float(revenue)
        
        ####### An additional operation on revenue is required based on currency
        
        dataframe = df_stock
        currency_reporting = dataframe[dataframe.companyid == companyid].reportingcurr
        
        #if currency_reporting.empty == True : 
            #currency_reporting = 0.0
            
        x = currency_reporting.to_string()
        x = x.split(' ')
        x = x[(len(x)-1)]
        currency_reporting = x
        
        #__________
        
        dataframe = df_stock
        currency_estimated = dataframe[dataframe.companyid == companyid].estcurr
        
        if currency_estimated.empty == True : 
            currency_estimated = 'Missing'
            
        x = currency_estimated.to_string()
        x = x.split(' ')
        x = x[(len(x)-1)]
        currency_estimated = x        
        

        #__________
        
        # for rate extraction; 
        # equivalency from fx to stock datasheets
        # base currency is considered = reporting currency
        # quot currency is considered = est currency
        
        # Logic
        
        # 1. 
        
        dataframe = df_fxrate
        rate = dataframe[(dataframe.basecur == currency_reporting) & (dataframe.quotcur == currency_estimated)].rate
        
        if rate.empty == True :
            
            rate = 0.01 # place holder if no rate is found from the fx dataframe
            
        rate = float(rate)
        
        # If reporting currency is different than estimated currency - df_stock
        
        if currency_reporting != currency_estimated:
            
            revenue = revenue * rate
            
        else:
            
            revenue = revenue
        
        
        ############################################################
        ############################################################
        # Output operations
        ############################################################
        ############################################################        
        
                       
        '''
        
        print('companyid',companyid)
        print(list_years)
        print('Roe generated for year', search_year)
        print('companyid', companyid)
        print('Roe generated for year', search_year)     
        print('toteqprev', total_equity_prev_year)
        print('companyid', companyid)
        print(search_year)
        print('companyid:', companyid)
        print('latestinfyear:', year)
        print('search_year:', search_year)
        print('revenue:',revenue)
        print('\n')
        '''
        
        list_revenue.append(revenue)
        
        
       
########## Dividing revenue list into chunks of 8 

# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
 
# How many elements each
# list should have
n = 8
 
list_revenue_chunks = list(divide_chunks(list_revenue, n))  


########## Inserting roe values in dfout

for i in dfout.index:
    
    chunk = list_revenue_chunks[i]
    
    dfout['revenue fy-4'][i] = chunk[0]
    dfout['revenue fy-3'][i] = chunk[1]
    dfout['revenue fy-2'][i] = chunk[2]
    dfout['revenue fy-1'][i] = chunk[3]
    dfout['revenue fy'][i]   = chunk[4]
    dfout['revenue fy+1'][i] = chunk[5]
    dfout['revenue fy+2'][i] = chunk[6]
    dfout['revenue fy+3'][i] = chunk[7]
        


In [47]:
dataframe = df_income
revenue = dataframe[(dataframe.companyid == 169) & (dataframe.year == 2016)].revenue
revenue
        

1085    7075.0
Name: revenue, dtype: float64

In [48]:
# Genering netde for latestfinyear

list_de = []

list_revenue = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    search_year = year
    
    de = 0 #prior to search, overwrite when de is calculated
    
    dataframe = df_balance
    total_debt = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totaldebt
    
    dataframe = df_balance
    cash_and_cash_equivalent = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].cashncashequiv
    
    dataframe = df_balance
    total_equity = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totequity
    
    
    # Check if any of the numerica values come out empty or missing
    
    if total_debt.empty == True:
        
        de = 0.01
        
    if cash_and_cash_equivalent.empty == True:
        
        de = 0.01
        
    if total_equity.empty == True:
        
        de = 0.01
        
    
    total_debt = float(total_debt)
    cash_and_cash_equivalent = float(cash_and_cash_equivalent)
    total_equity = float(total_equity)
    
    # Formulae
    # netde[2020] = (totaldebt[2020] – cashncashequiv[2020])/ totequity[2020]
    
    de = (total_debt - cash_and_cash_equivalent) / total_equity
    
    list_de.append(de)


####################
# Add de values to df out

dfout['netde fy'] = list_de

In [19]:
# Generating revenue 2 yr cagr

################## generating revenue columns
# similar procedure to roe and roa


list_cagr_2year = [] 


for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    search_year = dfout.latestfinyear[i] # collectes the year for current row
    
    search_year_2 = search_year + 2
    
    
    cagr_2year = 0

    
    # Extracting values for cagr computation
    
    # Extracting revenue for latest year + 2 from forecast
    
    dataframe = df_forecast
    revenue_2 = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year_2)].revenue
    
  
    # Extracting revenue for latest year from balance sheet dataframe
    
    dataframe = df_income
    revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue
    
    #print('2yrev: ', revenue_2)
    #print('rev: ', revenue)
    #print('\n')
    #___________________________________
    
    if (revenue_2.empty == True) | (revenue.empty == True):
        
        cagr_2year = 0.01
        
    else:
        
        revenue_2 = float(revenue_2)
        revenue = float(revenue)
        
        cagr_2year = 100 * ((revenue_2 / revenue) ** (0.5) - 1)
    
    list_cagr_2year.append(cagr_2year)
    
    
dfout['revenue 2yr cagr'] = list_cagr_2year # inserting list into the column

    
#########################################################################################################

# Generating revenue 2 yr cagr

#revenue 3yr cagr = 100*[(revenue fy+3 / revenue fy)^(0.33) – 1]





In [49]:
# Generating revenue 3 yr cagr

################## generating revenue columns
# similar procedure to roe and roa



list_cagr_3year = []

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    search_year = dfout.latestfinyear[i] # collectes the year for current row
    
    search_year_3 = search_year + 3
    
    cagr_3year = 0
    
    # Extracting values for cagr computation
   
    # Extracting revenue for latest year + 3 from forecast
    
    dataframe = df_forecast
    revenue_3 = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year_3)].revenue
    
    # Extracting revenue for latest year from balance sheet dataframe
    
    dataframe = df_income
    revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue
    
    #print('2yrev: ', revenue_2)
    #print('rev: ', revenue)
    #print('\n')
    
    #___________________________
    
    if (revenue_3.empty == True) | (revenue.empty == True):
        
        cagr_3year = 0.01
        
    else:
        
        revenue_3 = float(revenue_3)
        revenue = float(revenue)
        
        cagr_3year = 100 * ((revenue_3 / revenue) ** (0.33) - 1)
    
    list_cagr_3year.append(cagr_3year)
    
    

dfout['revenue 3yr cagr'] = list_cagr_3year # inserting list into the column
    
#########################################################################################################

# Generating revenue 2 yr cagr

#revenue 3yr cagr = 100*[(revenue fy+3 / revenue fy)^(0.33) – 1]





# dfout display

In [20]:
dfout

,companyid,latestfinyear,roe fy-4,roe fy-3,roe fy-2,roe fy-1,roe fy,roe fy+1,roe fy+2,roe fy+3,5yr median roe,roa fy-4,roa fy-3,roa fy-2,roa fy-1,roa fy,roa fy+1,roa fy+2,roa fy+3,5yr median roa,netde fy,revenue fy-4,revenue fy-3,revenue fy-2,revenue fy-1,revenue fy,revenue fy+1,revenue fy+2,revenue fy+3,revenue 2yr cagr,revenue 3yr cagr
0,30,2020,22.344145,18.42584,15.553547,19.794201,21.073279,19.3,18.5,16.4,19.794201,18.057368,15.043755,13.004084,16.972248,14.316593,NaN,NaN,NaN,15.043755,0.290795,4685988.0,4879559.0,5472882.0,6223057.0,5967362.0,6171000.0,6673000.0,7183000.0,5.747320,NaN
1,156,2020,8.448742,8.976136,8.873012,7.910407,-2.866362,0.0,0.0,0.0,8.448742,7.270872,7.600413,7.199134,6.216429,-2.147369,0.0,0.0,0.0,7.199134,0.286957,1801.95127,1923.12442,2072.03462,2408.97952,922.55382,0.01,0.01,0.01,0.010000,NaN
2,20,2020,11.284448,8.857234,8.970148,5.894449,-3.031918,-1.14,1.91,3.55,8.857234,7.019409,6.149829,6.789105,4.375199,-2.198656,-0.24,1.84,3.12,6.149829,0.087505,4796096.0,4203846.0,4218702.0,4047691.0,2046660.0,2098000.0,2631000.0,2952000.0,13.380293,NaN
3,169,2020,22.493225,15.047259,24.313187,23.179454,16.255443,12.91915,13.82389,13.98233,22.493225,14.376443,9.932618,15.915477,12.33564,8.796634,7.63626,8.32534,8.72676,12.33564,0.190851,7075.0,7769.0,8415.0,8776.0,8263.0,10187.83936,11380.04954,12529.1203,17.355433,NaN
4,37,2021,96.240114,15.632568,44.029164,27.584048,15.888524,15.25,14.95,15.6,27.584048,74.099225,10.705056,22.431093,13.934738,10.103292,13.95,13.9,16.6,13.934738,-0.370714,549.85451,541.7661,680.80776,663.69155,626.1228,679.15,728.3,823.4,7.851303,NaN
5,180,2020,36.145424,36.608498,31.888393,31.229987,30.211457,29.27112,30.70031,31.08441,31.888393,10.742409,11.048612,9.81813,10.014855,9.603284,10.26707,11.41877,12.14157,10.014855,0.250650,172092.5747,185803.9587,191996.8375,209771.8399,217897.1394,227493.4006,246125.9207,264634.1208,6.280334,NaN
6,22,2020,6.082263,5.233822,3.175776,3.657078,1.155763,0.3,1.6,3.8,3.657078,2.701403,2.385919,1.491362,1.734442,0.503727,0.1,0.7,1.6,1.734442,0.603178,2276459.607,2202820.511,2552536.173,2736388.551,2891388.396,2520000.0,2590000.0,2782000.0,-5.355220,NaN
7,124,2021,18.594323,10.970953,6.716376,2.138393,7.928588,14.61429,17.4,20.76,7.928588,7.894319,4.57017,2.80074,0.930307,3.696595,NaN,NaN,NaN,3.696595,0.406369,17308.615,17127.073,17150.54,16100.71,16504.521,19549.25,22751.11111,26410.5,17.408582,NaN
8,171,2021,2.998035,0.308594,5.544064,-5.59604,13.46159,13.4,13.48,13.63333,2.998035,1.488067,0.146518,2.501473,-1.877503,3.938465,5.0,5.04,5.275,1.488067,1.654687,605.441,639.741,676.26526,632.939,717.266,808.16,879.76,948.8,10.749553,NaN
9,17,2021,11.275374,12.488205,12.518209,9.44115,5.41752,9.95,11.55,0.0,11.275374,5.894345,6.602747,6.656175,5.15299,3.10193,NaN,NaN,0.0,5.894345,0.402422,27566.337,31043.651,35666.727,35803.038,30087.417,36319.5,40879.0,0.01,16.562185,NaN


# dfoutexport



In [21]:
dfout.to_csv('dfout.csv')

# Testing

In [23]:
dataframe = df_income
dataframe[(dataframe.companyid == companyid) & (dataframe.year == year)]


,companyid,year,revenue,incometaxexpense,netprofit
1324,191,2020,4839058.0,61223.0,-873181.0


In [24]:
len(dfout)

134

# Reference DFs

In [ ]:
df_balance[(df_balance.companyid == 172) & (df_balance.year == 2019)]

In [ ]:
df_income[df_income.companyid == 163]

In [ ]:
df_income.info()

In [ ]:
df_demo

In [ ]:
df_forecast[df_forecast.companyid == 30]

In [ ]:
dataframe = df_forecast
revenue_from_forecast = dataframe[(dataframe.companyid == 30) & (dataframe.year == 2021)].revenue
revenue_from_forecast

In [ ]:
df_fxrate[df_fxrate.basecur == 'CNY']

In [ ]:
df_stock

In [ ]:
df_stock.isna().sum()

In [ ]:
df_fxrate.isna().sum()